In [1]:
from nemo.collections.asr.models.aed_multitask_models import EncDecMultiTaskModel, MultiTaskTranscriptionConfig
import librosa
from IPython.display import Audio

/home/tassilo-holtzwart/Projects/Sotalis/stt_export/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2026-01-08 00:21:20 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.
OneLogger: Setting error_handling_strategy to DISABLE_QUIETLY_AND_REPORT_METRIC_ERROR for rank (rank=0) with OneLogger disabled. To override: explicitly set error_handling_strategy parameter.
No exporters were provided. This means that no telemetry data will be collected.


In [2]:
canary:EncDecMultiTaskModel = EncDecMultiTaskModel.from_pretrained(model_name="nvidia/canary-180m-flash", map_location="cpu").eval() # type: ignore

[NeMo I 2026-01-08 00:21:22 nemo_logging:393] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1152 tokens
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-08 00:21:22 nemo_logging:393] Aggregate vocab size: 5248


[NeMo W 2026-01-08 00:21:22 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 8
    pin_memory: true
    prompt_format: canary2
    max_tps: 25
    max_duration: 40.0
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    bucket_duration_bins:
    - - 3.56
      - 30
    - - 3.56
      - 77
    - - 4.608
      - 38
    - - 4.608
      - 88
    - - 5.48
      - 49
    - - 5.48
      - 106
    - - 6.05
      - 52
    - - 6.05
      - 109
    - - 6.85
      - 54
    - - 6.85
      - 124
    - - 7.914
      - 59
    - - 7.914
      - 137
    - - 8.52
      - 67
    - - 8.52
      - 158
    - - 9.51
      - 67
    - - 9.51
      - 153
    - - 10.29
      -

[NeMo I 2026-01-08 00:21:22 nemo_logging:393] PADDING: 0


Error getting class at nemo.collections.asr.modules.transformer.get_nemo_transformer: Located non-class of type 'function' while loading 'nemo.collections.asr.modules.transformer.get_nemo_transformer'


[NeMo I 2026-01-08 00:21:24 nemo_logging:393] Model EncDecMultiTaskModel was successfully restored from /home/tassilo-holtzwart/.cache/huggingface/hub/models--nvidia--canary-180m-flash/snapshots/b12ab418510d093e83890178fd0e8b0d0f7918a6/canary-180m-flash.nemo.


In [3]:
import os
audio_paths = [os.path.join(root, file)
               for root, _, files in os.walk("audios")
               for file in files if file.endswith(('.wav', '.mp3'))]

In [4]:
import torch

audio_batch = [torch.tensor(librosa.load(ap, sr= 16000)[0]) for ap in audio_paths]

In [5]:
audio_lengths = [len(audio) for audio in audio_batch]

In [6]:
audios = torch.nn.utils.rnn.pad_sequence(audio_batch, batch_first=True)
lengths = torch.tensor(audio_lengths, dtype=torch.int64)

In [8]:
canary.transcribe(audio_paths, source_lang='de', target_lang='de')

[NeMo W 2026-01-08 00:21:25 nemo_logging:405] Chunking is disabled. Please pass a single audio file or set batch_size to 1
[NeMo W 2026-01-08 00:21:25 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: trim_silence,enable_chunking
[NeMo W 2026-01-08 00:21:25 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 2it [00:05,  2.63s/it]


[Hypothesis(score=0.0, y_sequence=tensor([2215, 2359, 2505, 2218, 2252, 2212, 2529, 2178, 2609, 2728, 2390, 2212,
         2184, 2267, 2255, 2204, 2474, 2178, 2243, 2200, 2218, 2403, 2205, 2351,
         2440, 2184, 2459, 2866, 3062, 2327, 2611, 2197, 2955, 2400, 3054, 2218,
         2389, 2247, 2285, 2242, 2352, 2529, 2177, 2335, 2270, 2348, 2194, 2474,
         2763, 2228, 2299, 2290, 3075, 2223, 3051, 2412, 2213]), text=' Sadik , der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .', dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None),
 Hypothesis(score=0.0, y_sequence=tensor([2284, 2267, 2275, 2222, 2296, 2308, 2291, 2629, 2178, 2232, 2256, 2267,
         2965, 2194, 2262, 2213, 2569,

In [9]:
trcfg = MultiTaskTranscriptionConfig(
            batch_size=4,
            return_hypotheses=False,
            num_workers=0,
            channel_selector=None,
            augmentor=None,
            verbose=True,
            prompt={
                "source_lang": "de",
                "target_lang": "de"
            },
            timestamps=False,
        )

In [10]:
import warnings
prompt = [{'role': 'user', 'slots': {'source_lang': 'de', 'target_lang': 'de'}}]
default_turns = canary.prompt.get_default_dialog_slots()
turns = prompt.copy()  # shallow copy #1: don't override the config
for turn in turns:
    role = turn["role"]
    # Check if we have defaults for this role.
    # There shouldn't be more than a single turn for a given role, but if there are,
    # we'll emit a warning.
    if default_turns_for_role := [t for t in default_turns if t["role"] == role]:
        if len(default_turns_for_role) > 1:
            warnings.warn(
                f"More than one default turn detected for {role=}. "
                f"We'll be using default slot values for the first turn of {role=} only."
            )
        default_slots = default_turns_for_role[0]["slots"]
        turn["slots"] = turn["slots"].copy()  # shallow copy #1: don't override the config
        # fill missing slots using defaults
        for slot, val in default_slots.items():
            if turn["slots"].get(slot) is None:
                turn["slots"][slot] = val

decoder_input_ids = (
    canary.prompt.encode_dialog(turns=turns)["context_ids"]
    .unsqueeze(0)
    .repeat(len(audio_batch), 1)
    .to("cpu")
)

In [11]:
with torch.inference_mode():
    _, encoded_len, encoded, encoded_mask = canary(input_signal=audios, input_signal_length=lengths)

In [12]:
torch.save(encoded, "encoded8.pt")

In [13]:
canary.change_decoding_strategy({"strategy":"greedy", "return_best_hypothesis":True})

[NeMo I 2026-01-08 00:21:36 nemo_logging:393] Changed decoding strategy to 
    strategy: greedy
    compute_hypothesis_token_set: false
    preserve_alignments: null
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      tdt_include_duration: false
      method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
    compute_langs: false
    greedy:
      temperature: null
      max_generation_delta: -1
      preserve_alignments: false
      preserve_token_confidence: false
      confidence_method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
      n_samples: 1
    beam:
      beam_size: 1
      search_type: default
      len_pen: 1.0
      max_generation_delta: -1
      return_best_

In [14]:
hypothesis, = canary.decoding.decoding(encoder_hidden_states=encoded, encoder_input_mask = encoded_mask, decoder_input_ids=decoder_input_ids)

In [15]:
canary.decoding.decode_hypothesis(hypothesis)

[Hypothesis(score=0.0, y_sequence=tensor([2215, 2359, 2505, 2218, 2552, 2212, 2529, 2178, 2609, 2728, 2390, 2212,
         2184, 2267, 2255, 2204, 2474, 2178, 2243, 2200, 2218, 2403, 2205, 2351,
         2440, 2184, 2459, 2866, 3062, 2327, 2611, 2197, 2955, 2400, 3054, 2218,
         2389, 2247, 2285, 2242, 2352, 2529, 2177, 2335, 2270, 2348, 2194, 2474,
         2763, 2228, 2299, 2290, 3075, 2223, 3051, 2412, 2213]), text=' Sadik , Der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .', dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None),
 Hypothesis(score=0.0, y_sequence=tensor([2284, 2267, 2275, 2222, 2296, 2308, 2291, 2629, 2178, 2232, 2256, 2267,
         2965, 2194, 2262, 2213, 2569,

## Export

In [16]:
from export import exportable_forward, stft
import types


canary.preprocessor.featurizer.forward = types.MethodType(exportable_forward, canary.preprocessor.featurizer)
canary.preprocessor.featurizer.stft = types.MethodType(stft, canary.preprocessor.featurizer)

In [17]:
canary.preprocessor.export(
    "canary/preprocessor.onnx"
)

[NeMo I 2026-01-08 00:21:41 nemo_logging:393] Successfully exported AudioToMelSpectrogramPreprocessor to canary/preprocessor.onnx


(['canary/preprocessor.onnx'],
 ['nemo.collections.asr.modules.audio_preprocessing.AudioToMelSpectrogramPreprocessor exported to ExportFormat.ONNX'])

In [18]:
canary.encoder.export("canary/encoder.onnx")

[NeMo I 2026-01-08 00:21:45 nemo_logging:393] Successfully exported ConformerEncoder to canary/encoder.onnx


(['canary/encoder.onnx'],
 ['nemo.collections.asr.modules.conformer_encoder.ConformerEncoder exported to ExportFormat.ONNX'])

In [19]:
batch_size, seq_len, _ = canary.encoder.input_example()[0].shape

In [20]:
import torch

example_input = torch.randn(batch_size, seq_len, 512)
dynamic_axes = {"encoder_output": {0: "batch_size", 1: "seq_len"}}
torch.onnx.export(
    canary.encoder_decoder_proj,
    (example_input,),
    "canary/encoder_decoder_proj.onnx",
    dynamic_axes=dynamic_axes,
    input_names = ["encoder_output"],
    output_names = ["proj_output"],
    opset_version=17
)

# Decoder Export

In [21]:
encoded = torch.load("encoded8.pt", weights_only=False)

In [22]:
print(encoded.shape)

torch.Size([8, 129, 1024])


In [23]:
import types
def _exportable_get_memory_states(self, decoder_states, decoder_mems, i=0):
        # Access memory for the i-th layer
        past_mem = decoder_mems[i]
        # Concatenate memory and current states.
        # torch.cat handles empty tensors (seq_len=0) correctly in PyTorch/ONNX.
        return torch.cat((past_mem, decoder_states), dim=1)

canary.transf_decoder.decoder._get_memory_states = types.MethodType(_exportable_get_memory_states, canary.transf_decoder.decoder)


In [24]:
from nemo.collections.common.parts.transformer_utils import form_attention_mask

def exportable_forward(
    self,
    decoder_states,
    decoder_mask,
    encoder_states,
    encoder_mask,
    decoder_mems,
):
    # 1. Create masks
    # Ensure form_attention_mask uses torch operations (no numpy/python loops)
    decoder_attn_mask = form_attention_mask(decoder_mask, diagonal=self.diagonal)
    encoder_attn_mask = form_attention_mask(encoder_mask)

    # 2. Process layers
    new_mems = []

    # Layer 0: Embedding memory
    memory_states = self._get_memory_states(decoder_states, decoder_mems, 0)
    new_mems.append(memory_states.unsqueeze(0))

    curr_states = decoder_states
    for i, layer in enumerate(self.layers):
        # print(f"curr_states: {curr_states[:2,:2,:2]}")
        # print(f"decoder_attn_mask: {decoder_attn_mask[:2,:2,:2]}")
        # print(f"memory_states: {memory_states[:2,:2,:2]}")
        # print(memory_states.shape)

        curr_states = layer(curr_states, decoder_attn_mask, memory_states, encoder_states, encoder_attn_mask)
        # print(f"curr_states: {curr_states[:2,:2,:2]}")
        # print(f"curr_states shape: {curr_states.shape}")
        # Memory for layer i+1
        memory_states = self._get_memory_states(curr_states, decoder_mems, i + 1)
        new_mems.append(memory_states.unsqueeze(0))

    if self.final_layer_norm is not None:
        curr_states = self.final_layer_norm(curr_states)
        memory_states = self._get_memory_states(curr_states, decoder_mems, len(self.layers) + 1)
        new_mems.append(memory_states.unsqueeze(0))

    # Concatenate all memories into one tensor for the output
    # Shape: [6, B, L_total, H]
    combined_mems = torch.cat(new_mems,dim=0)
    return combined_mems


canary.transf_decoder.decoder.forward = types.MethodType(exportable_forward, canary.transf_decoder.decoder)

In [25]:
class TransformerDecoderWrapper(torch.nn.Module):
    def __init__(self, canary):
        super().__init__()
        self.decoder = canary.transf_decoder.decoder
        self.embedding = canary.transf_decoder.embedding
        self.num_states = canary.transf_decoder.num_states
        self.token_classifier = canary.decoding.log_softmax_module
    def forward(self,
    input_ids,      # [B, CurrSeqLen]
    decoder_mask,   # [B, TotalSeqLen]
    encoder_embeddings,
    encoder_mask,
    decoder_mems,   # [num_states, B, MemLen, H]
    start_pos,      # scalar tensor
    ):
        # Pass start_pos as a tensor to avoid it being optimized away as a constant
        decoder_embeddings = self.embedding.forward(input_ids=input_ids, start_pos=start_pos)
        # Decode
        combined_mems = self.decoder(
            decoder_states=decoder_embeddings,
            decoder_mask=decoder_mask,
            encoder_states=encoder_embeddings,
            encoder_mask=encoder_mask,
            decoder_mems=decoder_mems,
        )
        with self.token_classifier.with_log_softmax_enabled(False) as clf:
            logits = clf(hidden_states = combined_mems[-1][:, -1:])
        return logits, combined_mems

In [26]:
wrapper = TransformerDecoderWrapper(canary).eval()

In [27]:
decoded_sequence = []
input_ids = decoder_input_ids.clone()
memories = torch.zeros((6, len(audio_batch), 0, 1024))
pad_id = 2
eos_id = 3
pad_profile = torch.zeros(len(audio_batch),1) # which batched elements are already finished

with torch.inference_mode():
    for i in range(80):
        decoder_mask = (input_ids != pad_id).float()
        logits, memories = wrapper(input_ids= input_ids, decoder_mask=decoder_mask, encoder_embeddings=encoded, encoder_mask = encoded_mask, decoder_mems = memories, start_pos = torch.tensor(i))
        input_ids = logits[:,-1].argmax(dim=-1, keepdim=True)

        eos_reached = (input_ids == eos_id)
        pad_profile[eos_reached.squeeze(-1)] = 1
        if pad_profile.sum() == len(audio_batch):
            break
        decoded_sequence.append(input_ids)
sequences = torch.hstack(decoded_sequence)

In [28]:
import re

for sequence in sequences:
    text_tokens = []
    for t in sequence:
        if t.item() == eos_id:
            break
        text_tokens.append(t.item())
    text = canary.tokenizer.ids_to_text(text_tokens)
    # we always have space before punctuation, remove it
    text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text)
    print(text)

 Sadik, Der in ihren Herzen wie in seinem eigenen las, hatte immer seine Freude daran gehabt, dem Wachstum ihrer gegenseitigen Neigung zuzusehen.
 Kein Falsch ist an ihnen und kein Flitter. Ein Wort, eine Enziansblüte im Gebirge.
 Vierzig Jahre war Conrad Ferdinand Mayer, als er sein erstes Buch, ein kleines Buch Gedichte, veröffentlichte.
 die so glücklich sind, keinen anderen Gesetzgeber zu kennen als die Natur.
 Was wollten wir machen, wenn er ihrer zwei oder drei mit sich nähme? Er würde es immer so anzugehen wissen, dass er am Ende recht behielte.
 Was ist mit Mister Rochester geschehen? Wie geht es ihm? Wo ist er
 Dann näherte er sich dem Feuer. Es war ein hartes Stück Arbeit hierher zu gelangen. Das kann ich Sie versichern.
 Ja. Wiederum entstand eine Pause. Die Uhr schlug acht Schläge.


In [29]:
def export_decoder_to_onnx(model, onnx_path):
    batch_size = 1
    enc_seq_len = 10
    num_states = 6
    hidden_dim = 1024

    # Dummy inputs
    input_ids = torch.ones((batch_size, 1), dtype=torch.long)
    decoder_mask = torch.ones((batch_size, 1), dtype=torch.float32)
    encoder_embeddings = torch.randn((batch_size, enc_seq_len, hidden_dim))
    encoder_mask = torch.ones((batch_size, enc_seq_len), dtype=torch.float32)
    # Use sequence length 1 instead of 0 for dummy inputs to ensure graph logic is traced
    decoder_mems = torch.zeros((num_states, batch_size, 1, hidden_dim))
    start_pos = torch.tensor(0, dtype=torch.long)

    dynamic_axes = {
        "input_ids": {0: "batch_size", 1: "seq_len"},
        "decoder_mask": {0: "batch_size", 1: "total_seq_len"},
        "encoder_embeddings": {0: "batch_size", 1: "enc_seq_len"},
        "encoder_mask": {0: "batch_size", 1: "enc_seq_len"},
        "decoder_mems": {1: "batch_size", 2: "mem_len"},
        "logits": {0: "batch_size"},
        "new_mems": {1: "batch_size", 2: "total_seq_len"},
    }

    torch.onnx.export(
        model,
        (input_ids, decoder_mask, encoder_embeddings, encoder_mask, decoder_mems, start_pos),
        onnx_path,
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=["input_ids", "decoder_mask", "encoder_embeddings", "encoder_mask", "decoder_mems", "start_pos"],
        output_names=["logits", "new_mems"],
        dynamic_axes=dynamic_axes,
    )

In [30]:
export_decoder_to_onnx(wrapper, "canary/decoder.onnx")

# ONNX inference

In [31]:
import onnxruntime as ort

In [32]:
encoder_session = ort.InferenceSession("canary/encoder.onnx")
preprocessor_session = ort.InferenceSession("canary/preprocessor.onnx")
proj_session = ort.InferenceSession("canary/encoder_decoder_proj.onnx")
decoder_session = ort.InferenceSession("canary/decoder.onnx")

In [33]:
import librosa
import numpy as np

audio, sr = librosa.load("test_audio.wav", sr=16000)


In [34]:
preprocessed_np, length_np = preprocessor_session.run(None, {"input_signal": audio.astype(np.float32).reshape(1, -1), "length": np.array([len(audio)], dtype=np.int64)})

In [35]:
encoded_np, encoded_length_np = encoder_session.run(None, {"audio_signal": preprocessed_np, "length": length_np})

In [37]:
projected, = proj_session.run(None, {"encoder_output": encoded_np.transpose(0,2,1)})


In [56]:
projected.shape

(1, 138, 1024)

In [58]:
# create a mask using the encoded_length_np tensor that has shape like projected at [0,1]
encoded_mask_np = np.arange(encoded_np.shape[2])[None, :] < encoded_length_np

In [60]:
# Decoder Loop
input_ids = decoder_input_ids.numpy()
memories = np.zeros((6, input_ids.shape[0], 0, 1024), dtype=np.float32)
pad_id, eos_id = 2, 3
decoded_tokens = []
finished = np.zeros(input_ids.shape[0], dtype=bool)

for i in range(80):
    # decoder_mask is only for the CURRENT token being processed, not the full history
    # The memory mechanism handles the historical context
    current_decoder_mask = (input_ids != pad_id).astype(np.float32)

    outputs = decoder_session.run(None, {
        "input_ids": input_ids.astype(np.int64),
        "decoder_mask": current_decoder_mask,
        "encoder_embeddings": projected,
        "encoder_mask": encoded_mask_np.astype(np.float32),
        "decoder_mems": memories,
        "start_pos": np.array(i, dtype=np.int64)
    })
    logits, memories = outputs[0], outputs[1]

    # Greedy selection
    next_tokens = np.argmax(logits[:, -1, :], axis=-1, keepdims=True)
    input_ids = next_tokens

    finished |= (next_tokens.flatten() == eos_id)
    if finished.all():
        break
    decoded_tokens.append(next_tokens)

result_ids = np.concatenate(decoded_tokens, axis=1)
for res in result_ids:
    print(canary.tokenizer.ids_to_text(res[res != eos_id].tolist()))

 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
 Es war in den siebziger Jahren an einem Dezember , also am Tage nach Sankt Nikolaus .
